In [2]:
!pip install spacy
!pip install newspaper3k

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.1/211.1 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.1/81.1 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.7/97.7 kB 14.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 68.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for tinysegmenter: filename=tinysegmenter-0.3-py3-none-any.whl size=13540 sha256=840a38c8d4a5ab65dc0299d53cee32391d567992c65540a7163fc314b4a7c339
  Stored in directory: /root/.cache/pip/wheels/c8/d6/6c/384f58df48c00b9a31d638005143b5b3ac62c3d25fb1447f23
  Created wheel for feedfinder2: filename=feedfinder2-0.0.4-py3-none-any.whl size=3339 sha256=adb5aceb5a6607729930fc7a919cdf2c32ec5f03edff9871d557364de45d1cc1
  Stored in directory: /root/.cache/pip/wheels/97/02/e7/a1ff1760e12bdbaab0ac824fae5c1

In [3]:
import spacy
import en_core_web_sm
from spacy.lang.en.stop_words import STOP_WORDS
from string import punctuation
from heapq import nlargest
from newspaper import Article


def generate_summary(url):
    url1 = url
    article = Article(url1)
    article.download()

    try:
        article.parse()
    except:
        pass

    rawtext = article.text
    stopwords = list(STOP_WORDS)

    nlp = en_core_web_sm.load()
    doc = nlp(rawtext)

    word_freq = {}
    for word in doc:
        if word.text.lower() not in stopwords and word.text.lower() not in punctuation:
            if word.text not in word_freq.keys():
                word_freq[word.text] = 1
            else:
                word_freq[word.text] += 1

    try:
        max_freq = max(word_freq.values())
    except:
        max_freq = 0

    for word in word_freq.keys():
        word_freq[word] = word_freq[word] / max_freq

    sen_tokens = [sent for sent in doc.sents]

    sent_scores = {}
    for sent in sen_tokens:
        for word in sent:
            if word.text in word_freq.keys():
                if sent not in sent_scores.keys():
                    sent_scores[sent] = word_freq[word.text]
                else:
                    sent_scores[sent] += word_freq[word.text]

    if len(rawtext.split()) > 1199:
        select_len = int(len(sen_tokens) * 0.05)
    elif 799 < len(rawtext.split()) < 1200:
        select_len = int(len(sen_tokens) * 0.08)
    elif 499 < len(rawtext.split()) < 800:
        select_len = int(len(sen_tokens) * 0.15)
    elif 199 < len(rawtext.split()) < 500:
        select_len = int(len(sen_tokens) * 0.30)
    elif len(rawtext.split()) < 200:
        select_len = int(len(sen_tokens) * 0.50)
    else:
        select_len = int(len(sen_tokens) * 0.20)

    summary = nlargest(select_len, sent_scores, key=sent_scores.get)
    final_summary = [word.text for word in summary]
    summary = ' '.join(final_summary)

    return summary

url2 = "https://www.thedailystar.net/news/bangladesh/elections/news/govt-can-only-be-changed-through-vote-pm-3472661"
result_summary = generate_summary(url2)
print(result_summary)



Denounces attempts to foil polls through violence, arson; urges countrymen to be resist such attempts

Prime Minister Sheikh Hasina today said the government can only be changed through a vote and denounced the efforts to foil the polls through violence and arson.

 "We want to protect the voting rights of the people ... and the government can only be changed through vote," she said while inaugurating the sale of AL nomination papers for the 12th parliamentary elections at the party's central office.

 Sheikh Hasina hoped that people would resist those who were trying to snatch away their voting rights unleashing arson terrorism.

 "Those political parties which don't have trust and confidence in the people, and which are not well organised, they are trying to foil the election," she said.


